## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dombas,NO,62.0696,9.1235,5.72,99,91,2.53,overcast clouds
1,1,Fuxin,CN,42.0156,121.6589,13.01,47,0,7.61,clear sky
2,2,Ermelo,ZA,-26.5333,29.9833,54.54,71,10,3.40,clear sky
3,3,Newport Beach,US,33.6189,-117.9289,63.09,74,75,1.01,broken clouds
4,4,Khatanga,RU,71.9667,102.5000,-37.68,99,20,2.37,few clouds


In [29]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [30]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Butaritari,KI,3.0707,172.7902,82.00,77,18,11.72,light rain
10,10,Orocue,CO,4.7912,-71.3354,83.82,34,100,5.35,overcast clouds
15,15,Bandarbeyla,SO,9.4942,50.8122,75.02,78,20,15.79,few clouds
17,17,Puerto Ayora,EC,-0.7393,-90.3518,75.54,96,100,3.00,overcast clouds
25,25,Wellington,NZ,-41.2866,174.7756,78.46,63,67,3.00,moderate rain


In [33]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()
#preferred_cities_df.isnull().sum()

City_ID                155
City                   155
Country                155
Lat                    155
Lng                    155
Max Temp               155
Humidity               155
Cloudiness             155
Wind Speed             155
Current Description    155
dtype: int64

In [34]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [35]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Butaritari,KI,82.00,light rain,3.0707,172.7902,
10,Orocue,CO,83.82,overcast clouds,4.7912,-71.3354,
15,Bandarbeyla,SO,75.02,few clouds,9.4942,50.8122,
17,Puerto Ayora,EC,75.54,overcast clouds,-0.7393,-90.3518,
25,Wellington,NZ,78.46,moderate rain,-41.2866,174.7756,
26,Saint-Joseph,RE,79.16,moderate rain,-21.3667,55.6167,
28,Atuona,PF,78.87,few clouds,-9.8000,-139.0333,
34,Souillac,MU,79.36,light intensity shower rain,-20.5167,57.5167,
35,Faanui,PF,80.94,few clouds,-16.4833,-151.7500,
39,Dingle,PH,79.77,overcast clouds,10.9995,122.6711,


In [36]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#     base_url = "https://maps.googleapis.com/maps/api/directions/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [37]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()


City                   155
Country                155
Max Temp               155
Current Description    155
Lat                    155
Lng                    155
Hotel Name             155
dtype: int64

In [38]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [43]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]


In [44]:
# 11a. Add a marker layer for each city to the map. 

max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=max_temp,
                                 dissipating=False, 
                                 max_intensity=300, 
                                 point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))